# 少しの工夫で早くなる

備考：スクリプトの書き方は，敢えて統一した書き方作法を行っておらず，様々な流儀で書いています。

### %timeitの使い方
https://ipython.readthedocs.io/en/stable/interactive/magics.html

In [2]:
n = 1000000
%timeit sum(range(n))

43.3 ms ± 2.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


上記は，10回ループを7回試行，1ループあたりの平均実行時間とその標準偏差が表示されている。<br>
次は，ループと試行の回数を明示的に指定する例。

In [3]:
%timeit -r10 -n20 sum(range(n))

41.6 ms ± 2.48 ms per loop (mean ± std. dev. of 10 runs, 20 loops each)


%timeitが一行だけの測定，これに対して%%timeitはセル全体の測定を行う。この使い方は，以下の例でわかるであろう。

### dis ； Python バイトコードの逆アセンブラ
https://docs.python.org/ja/3/library/dis.html <br>
https://docs.python.org/3/library/dis.html <br>

In [1]:
from dis import dis

In [5]:
def myfunc(alist):
    return len(alist)

dis(myfunc)

  2           0 LOAD_GLOBAL              0 (len)
              2 LOAD_FAST                0 (alist)
              4 CALL_FUNCTION            1
              6 RETURN_VALUE


### オーバーヘッドの例

In [6]:
%%timeit
def pow2(x):
    return x+x

x = 1
for i in range(1,11):
    x = pow2(x)

1.51 µs ± 73.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%%timeit
x = 1
x += x
x += x
x += x
x += x
x += x
x += x
x += x
x += x
x += x
x += x

299 ns ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


関数def pow2()を呼出すのにオーバーヘッドを要する，2番目のべたな書き方はこれを回避しているため，比較して早くなる。<br>
べき乗の数が大きければ，次が一番早い。

In [8]:
%timeit x = 2**10

15.9 ns ± 0.471 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


### 演算コード <<, + , * の比較
2倍の演算について，C言語やアセンブリ言語ではビットシフト（<<）が最も早いと言われているが，理由は不明であるがPythonは異なる。

In [9]:
%%timeit
for i in range(10000):
    x = 0b1111 ; x <<1

797 µs ± 43.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit
for i in range(10000):
    x = 0b1111 ; x = x*2

496 µs ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit
for i in range(10000):
    x = 0b1111 ; x = x+x

540 µs ± 57.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


それぞれのバイトコードを見る

In [12]:
def check_shift_multiple_add(a):
    x = a ; x <<1
    x = a ; x = x*2
    x = a ; x = x + 2

In [13]:
dis(check_shift_multiple_add)

  2           0 LOAD_FAST                0 (a)
              2 STORE_FAST               1 (x)
              4 LOAD_FAST                1 (x)
              6 LOAD_CONST               1 (1)
              8 BINARY_LSHIFT
             10 POP_TOP

  3          12 LOAD_FAST                0 (a)
             14 STORE_FAST               1 (x)
             16 LOAD_FAST                1 (x)
             18 LOAD_CONST               2 (2)
             20 BINARY_MULTIPLY
             22 STORE_FAST               1 (x)

  4          24 LOAD_FAST                0 (a)
             26 STORE_FAST               1 (x)
             28 LOAD_FAST                1 (x)
             30 LOAD_CONST               2 (2)
             32 BINARY_ADD
             34 STORE_FAST               1 (x)
             36 LOAD_CONST               0 (None)
             38 RETURN_VALUE


### リストの作成と初期化
リストを作るなら内包表現

In [2]:
%%timeit
N = 100000
[0]*N

413 µs ± 7.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
%%timeit
N = 100000
[0 for _ in range(N)]

3.68 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


備考：上記のアンダースコア"_"は，変数名と考えてよい。ただし，他の変数名を使っていないという意味。なお，変数名の先頭にアンダースコアを付けるとは，関数内のローカル変数であることを明示するときなどに使う。<br>
これは，次のPEP8にある。https://www.python.org/dev/peps/pep-0008/

##### 両者のバイトコードを見る

In [10]:
def make_list_1(n):
    [0]*n

def make_list_2(n):
    [0 for _ in range(n)]

dis(make_list_1)
#dis(make_list_2)

  2           0 LOAD_CONST               1 (0)
              2 BUILD_LIST               1
              4 LOAD_FAST                0 (n)
              6 BINARY_MULTIPLY
              8 POP_TOP
             10 LOAD_CONST               0 (None)
             12 RETURN_VALUE


### リストを作るなら内包表現

In [2]:
def test_list_1(n):
    xa = []
    for i in range(n):
        xa.append(i)

%timeit test_list_1(100000)

9.95 ms ± 468 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
#dis(test_list_1)

In [4]:
def test_list_2(n):
    xb = [0]*n
    for i in range(n):
        xb[i] = i

%timeit test_list_2(100000)

6.56 ms ± 812 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
#dis(test_list_2)

In [6]:
def test_list_3(n):
    xc = [i for i in range(n)]

%timeit test_list_3(100000)

4.07 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
#dis(test_list_3)

### while文とfor文の比較
while文の方が，for文よりも実行ステップ数が多い。 次の説明では，while文の実行ステップ数は10，for文は3である。 https://stackoverflow.com/questions/869229/why-is-looping-over-range-in-python-faster-than-using-a-while-loop <br>
大まかには，while文は i+= 1の処理に複数の実行ステップを要し，for文のrange()では数のインクリメントのみで済んでいる。

In [18]:
%%timeit
i = 0
while i < 10000000:
    i += 1

697 ms ± 48.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
for i in range(0, 10000000):
    pass

217 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


上記の2つのバイトコードを見るために，次を設ける。

In [22]:
def test_while_for_1(n):
    i = 0
    while i < n:
        i += 1

def test_while_for_2(n):
    for i in range(0,n):
        pass

In [24]:
dis(test_while_for_1)
#dis(test_while_for_2)

  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (i)

  3           4 SETUP_LOOP              20 (to 26)
        >>    6 LOAD_FAST                1 (i)
              8 LOAD_FAST                0 (n)
             10 COMPARE_OP               0 (<)
             12 POP_JUMP_IF_FALSE       24

  4          14 LOAD_FAST                1 (i)
             16 LOAD_CONST               2 (1)
             18 INPLACE_ADD
             20 STORE_FAST               1 (i)
             22 JUMP_ABSOLUTE            6
        >>   24 POP_BLOCK
        >>   26 LOAD_CONST               0 (None)
             28 RETURN_VALUE


### lambda関数は遅い  　

In [31]:
def test_calc(x):
    return x+x+x

In [32]:
%%timeit
for i in range(100000):
    y = test_calc(i)

16.3 ms ± 336 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [89]:
dis(test_calc)

  2           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                0 (x)
              4 BINARY_ADD
              6 LOAD_FAST                0 (x)
              8 BINARY_ADD
             10 RETURN_VALUE


In [33]:
%%timeit
for i in range(100000):
    y = (lambda x:x+x+x)(i)

23.5 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


lambda関数のバイトコードを見るため，次を設ける。

In [35]:
def test_lambda(n):
    for i in range(n):
        y = (lambda x:x+x+x)(i)

dis(test_lambda)

  2           0 SETUP_LOOP              28 (to 30)
              2 LOAD_GLOBAL              0 (range)
              4 LOAD_FAST                0 (n)
              6 CALL_FUNCTION            1
              8 GET_ITER
        >>   10 FOR_ITER                16 (to 28)
             12 STORE_FAST               1 (i)

  3          14 LOAD_CONST               1 (<code object <lambda> at 0x000002CF69109ED0, file "<ipython-input-35-07d25824da32>", line 3>)
             16 LOAD_CONST               2 ('test_lambda.<locals>.<lambda>')
             18 MAKE_FUNCTION            0
             20 LOAD_FAST                1 (i)
             22 CALL_FUNCTION            1
             24 STORE_FAST               2 (y)
             26 JUMP_ABSOLUTE           10
        >>   28 POP_BLOCK
        >>   30 LOAD_CONST               0 (None)
             32 RETURN_VALUE

Disassembly of <code object <lambda> at 0x000002CF69109ED0, file "<ipython-input-35-07d25824da32>", line 3>:
  3           0 LOAD_FAST      

### 数学関数で引数がスカラー値の場合
mathの方がNumPyの関数より速い。例えば、math.sin(x)はnumpy.sin(x)よりはるかに速い。  
Python Scripting for Computational Science, 2008 より

In [36]:
import numpy as np
import math

In [37]:
%timeit y = np.sin(1.0)

730 ns ± 20 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [38]:
%timeit y = math.sin(1.0)

162 ns ± 6.59 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
